# Exporting to a database using pybarb

In this demo we will show you how to pull data from the advertising spots endpoint and then export it to a database using pybarb.
Note the full API documentation can be found [here](https://barb-api.co.uk/api-docs). 

It might also be useful to consult the [Getting Started](https://barb-api.co.uk/api-docs#section/Getting-started) section for information about authentication and basic API usage.


## Querying the API with pybarb

First we will repeat the steps carried out in the video *Pulling data from the advertising spots endpoint using pybarb* to extract some new data.

In [7]:
import json
import pybarb as pb
import sqlalchemy as db

# Set the working directory
working_directory = '/path/to/your/dir/'

# Get the access token
with open(working_directory + "creds.json") as file:
    creds = json.load(file)

# Create a BarbAPI object and connect
barb_api = pb.BarbAPI(creds)
barb_api.connect()

spot_data = barb_api.advertising_spots(min_transmission_date = "2022-01-01",
                           max_transmission_date = "2022-04-01", 
                           buyer =  "mindshare_media_uk_l", 
                           advertiser = "UNILEVER_UK_HOME_&_P",
                           panel="BBC Network")

spot_df = spot_data.to_dataframe()
spot_df.head()


,panel_region,station_name,spot_type,spot_start_datetime,spot_duration,preceding_programme_name,succeeding_programme_name,break_type,position_in_break,broadcaster_spot_number,...,clearcast_buyer_code,clearcast_buyer_name,clearcast_advertiser_code,clearcast_advertiser_name,campaign_approval_id,sales_house_name,audience_size_hundreds,date_of_transmission,audience_name,audience_target_size_hundreds
0,BBC Network,Magic TV,spot,2022-03-18 21:36:35,30,TBA,TBA,centre break,other,180948555,...,B000925,MINDSHARE MEDIA UK L,A004642,UNILEVER UK HOME & P,1048980.0,Channel 4 Sales,0,2022-03-18,Men ABC1 16-34,41000
1,BBC Network,Magic TV,spot,2022-03-18 21:36:35,30,TBA,TBA,centre break,other,180948555,...,B000925,MINDSHARE MEDIA UK L,A004642,UNILEVER UK HOME & P,1048980.0,Channel 4 Sales,0,2022-03-18,Men 35-44,39910
2,BBC Network,Magic TV,spot,2022-03-18 21:36:35,30,TBA,TBA,centre break,other,180948555,...,B000925,MINDSHARE MEDIA UK L,A004642,UNILEVER UK HOME & P,1048980.0,Channel 4 Sales,0,2022-03-18,All Men,250100
3,BBC Network,Magic TV,spot,2022-03-18 21:36:35,30,TBA,TBA,centre break,other,180948555,...,B000925,MINDSHARE MEDIA UK L,A004642,UNILEVER UK HOME & P,1048980.0,Channel 4 Sales,0,2022-03-18,Adults 16-34,141860
4,BBC Network,Magic TV,spot,2022-03-18 21:36:35,30,TBA,TBA,centre break,other,180948555,...,B000925,MINDSHARE MEDIA UK L,A004642,UNILEVER UK HOME & P,1048980.0,Channel 4 Sales,0,2022-03-18,Adults C2,108040


## Connecting to a database and exporting the data

For the purposes of this demonstration we will create a simple SQLite database and export the data to it. SQLite is a lightweight database that is ideal for testing and prototyping. 

pybarb uses SQLAlchemy to connect to databases. SQLAlchemy is a Python SQL toolkit and Object Relational Mapper (ORM) that gives application developers the full power and flexibility of SQL. It can be used with any database supported by SQLAlchemy. Some of the most popular databases supported by SQLAlchemy are:

* PostgreSQL
* MySQL
* SQLite
* Oracle
* Microsoft SQL Server

For more information about SQLAlchemy see the [documentation](https://docs.sqlalchemy.org/en/13/).

In [8]:
# Define the database connection
DATABASE_URL = 'sqlite:///example.db'
engine = db.create_engine(DATABASE_URL, echo=False)


Now we export the data to a database. We will use the SQLAlchemy SQLite dialect to connect to a SQLite database. We will also use the SQLAlchemy ORM to create a table in the database and insert the data into it.


In [9]:
spot_data.to_sql(connection_string=DATABASE_URL, table_name='spot_data')

To prove that it worked let's connect to the database and query the table.

In [10]:
connection = engine.connect()
metadata = db.MetaData()
spot_data_table = db.Table('spot_data', metadata, autoload_with=engine)
query = spot_data_table.select()
result = connection.execute(query).fetchall()
result[:10]


[('BBC Network', 'Magic TV', 'spot', datetime.datetime(2022, 3, 18, 21, 36, 35), 30, 'TBA', 'TBA', 'centre break', 'other', '180948555', 'TAGDGGB071030', 'Persil Gaia 30 NEW', 0, 'England, Scotland and Wales', 'B000925', 'MINDSHARE MEDIA UK L', 'A004642', 'UNILEVER UK HOME & P', 1048980.0, 'Channel 4 Sales', 0, datetime.date(2022, 3, 18), 'Men ABC1 16-34', 41000),
 ('BBC Network', 'Magic TV', 'spot', datetime.datetime(2022, 3, 18, 21, 36, 35), 30, 'TBA', 'TBA', 'centre break', 'other', '180948555', 'TAGDGGB071030', 'Persil Gaia 30 NEW', 0, 'England, Scotland and Wales', 'B000925', 'MINDSHARE MEDIA UK L', 'A004642', 'UNILEVER UK HOME & P', 1048980.0, 'Channel 4 Sales', 0, datetime.date(2022, 3, 18), 'Men 35-44', 39910),
 ('BBC Network', 'Magic TV', 'spot', datetime.datetime(2022, 3, 18, 21, 36, 35), 30, 'TBA', 'TBA', 'centre break', 'other', '180948555', 'TAGDGGB071030', 'Persil Gaia 30 NEW', 0, 'England, Scotland and Wales', 'B000925', 'MINDSHARE MEDIA UK L', 'A004642', 'UNILEVER UK HO

We can also use SQLAlchemy to append data to an existing table. In this example we will append the data to the table we created in the previous example.


In [11]:
spot_data.to_sql(connection_string=DATABASE_URL, table_name='spot_data', if_exists='append')

You might for example have a job set up to export the data to your database every day. You could then use the database to create a dashboard to show the client how the campaign is performing.
